# Oscar Wilde's Use of Literary Devices Over Time
##### Jessica Brown and Lauren Nalajala
## Background
Talk about Oscar Wilde's life here

And the goal of this project here

## Intial Text Processing
Talk about getting the data, processing it (ie. what we removed)

## Word Usage
Talk about how we got the most common words

In [ ]:
# create plots of his most commonly used words in one book, and focus in on specific words here?